<a href="https://colab.research.google.com/github/sh-0620/dacon-recommendation-system/blob/main/%08tf-idf_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings

from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
data_path = '/content/drive/MyDrive/데이콘/기사추천시스템/open/'

In [62]:
article_info = pd.read_csv(data_path + 'article_info.csv')

In [63]:
article_info #기사 정보(userid = 기사 작성자)

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion
0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN
...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - C...,"Quando iniciei minha jornada em Swift, saindo ...",HTML,pt,USER_0882,BR,MG
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,It's been an amazing journey to get here and i...,HTML,en,USER_0220,NaN,NaN
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin ...,This is part one in a series of posts about Ko...,HTML,en,USER_1010,BR,SP
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a dist...,"O Instituto Tecnológico de Aeronáutica (ITA) ,...",HTML,pt,USER_1210,NaN,NaN


In [64]:
article_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   articleID    3008 non-null   object
 1   Title        3008 non-null   object
 2   Content      3008 non-null   object
 3   Format       3008 non-null   object
 4   Language     3008 non-null   object
 5   userID       3008 non-null   object
 6   userCountry  659 non-null    object
 7   userRegion   657 non-null    object
dtypes: object(8)
memory usage: 188.1+ KB


In [127]:
article_info

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion
0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN
...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - C...,"Quando iniciei minha jornada em Swift, saindo ...",HTML,pt,USER_0882,BR,MG
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,It's been an amazing journey to get here and i...,HTML,en,USER_0220,NaN,NaN
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin ...,This is part one in a series of posts about Ko...,HTML,en,USER_1010,BR,SP
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a dist...,"O Instituto Tecnológico de Aeronáutica (ITA) ,...",HTML,pt,USER_1210,NaN,NaN


In [65]:
view_log = pd.read_csv(data_path + 'view_log.csv')

In [66]:
view_log #사용자-기사 데이터

,userID,articleID,userRegion,userCountry
0,USER_0000,ARTICLE_0661,NY,US
1,USER_0000,ARTICLE_2316,NY,US
2,USER_0000,ARTICLE_1345,NY,US
3,USER_0000,ARTICLE_1089,NY,US
4,USER_0000,ARTICLE_1484,NY,US
...,...,...,...,...
42712,USER_1420,ARTICLE_0682,SP,BR
42713,USER_1420,ARTICLE_2179,SP,BR
42714,USER_1420,ARTICLE_1848,SP,BR
42715,USER_1420,ARTICLE_0030,SP,BR


In [68]:
view_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42717 entries, 0 to 42716
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   userID       42717 non-null  object
 1   articleID    42717 non-null  object
 2   userRegion   42717 non-null  object
 3   userCountry  42717 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

In [112]:
# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000) #stop_words='english'는 영어 불용어를 제거하고, max_features=5000는 최대 5000개의 단어 특성을 사용한다는 의미입니다.
tfidf_matrix = tfidf_vectorizer.fit_transform(article_info['Title'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [113]:
def get_personalized_recommendations(user_id, cosine_sim=cosine_sim, articles_df=article_info):
    # 사용자가 읽은 기사 목록 가져오기
    user_articles = view_log[view_log['userID'] == user_id]['articleID'].tolist()

    # 사용자가 읽은 기사의 인덱스 가져오기
    article_indices = article_info[article_info['articleID'].isin(user_articles)].index.tolist()

    # 유사도 점수 합산
    sim_scores = cosine_sim[article_indices].sum(axis=0)

    # 유사도 점수를 기준으로 기사 정렬
    ranked_indices = sim_scores.argsort()[::-1]

    # 상위 5개 기사 추천
    recommended_article_indices = ranked_indices[:5]

    return articles_df.iloc[recommended_article_indices]



In [103]:
get_personalized_recommendations('USER_1420')['articleID'].values

array(['ARTICLE_1351', 'ARTICLE_1891', 'ARTICLE_1788', 'ARTICLE_0445',
       'ARTICLE_2409'], dtype=object)

In [114]:
recommendations = []
for idx, user in enumerate(view_log.userID.unique()):
    recommendations.append([user, get_personalized_recommendations(user).articleID.values])

In [115]:
recommendations = pd.DataFrame(recommendations, columns=['userID', 'articleID'])

In [116]:
recommendations = recommendations.explode('articleID').reset_index(drop=True)


In [117]:
submission = recommendations.to_csv(data_path + 'submission_v2.csv')

In [118]:
submission

In [111]:
data_path

'/content/drive/MyDrive/데이콘/기사추천시스템/open/'

# word2vec

In [121]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [123]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [134]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# 예시 데이터프레임 생성
data = article_info


# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자 변환
    text = text.lower()
    # 구두점 제거
    text = text.translate(str.maketrans('', '', string.punctuation))
    # 불용어 제거 (예시로 nltk의 불용어 사용)
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

# 텍스트 전처리 및 단어 리스트 생성
processed_texts = [preprocess_text(text) for text in article_info['Content']]

# Word2Vec 모델 학습
model = Word2Vec(processed_texts, vector_size=100, window=5, min_count=1, sg=1)

# 기사 벡터화
# 기사 제목 벡터화

# 기사 벡터화 및 유사도 계산 예시
def calculate_article_similarity(article1, article2, model):
    vec1 = model.wv[article1]
    vec2 = model.wv[article2]
    similarity = cosine_similarity([vec1], [vec2])[0][0]
    return similarity

# 예시: ARTICLE_2441과 ARTICLE_1270의 유사도 계산
similarity_score = calculate_article_similarity('article', 'article', model)
print(f"Similarity between ARTICLE_2441 and ARTICLE_1270: {similarity_score}")


# 개인별 기사 추천 함수 정의
def recommend_articles(user_id, model, article_info, top_n=5):
    # 사용자가 읽은 기사ID 목록
    read_articles = article_info[article_info['userID'] == user_id]['articleID'].tolist()

    # 사용자가 읽지 않은 기사ID 목록
    unread_articles = article_info[~article_info['articleID'].isin(read_articles)]['articleID'].unique()

    # 사용자가 읽지 않은 기사들과의 유사도 계산
    similarities = []
    for article_id in unread_articles:
        similarity = 0
        for read_article_id in read_articles:
            similarity += calculate_article_similarity(read_article_id, article_id, model)
        similarity /= len(read_articles)  # 평균 유사도 계산
        similarities.append((article_id, similarity))

    # 유사도가 높은 순서로 정렬
    similarities.sort(key=lambda x: x[1], reverse=True)

    # 상위 top_n개 기사 추천
    recommended_articles = similarities[:top_n]

    return recommended_articles

Similarity between ARTICLE_2441 and ARTICLE_1270: 1.0


In [135]:
recommendations = recommend_articles('USER_0000', model, article_info)

KeyError: "Key 'ARTICLE_1345' not present"